In [260]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import gpboost as gpb
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Gathering the data

In [261]:
path = kagglehub.dataset_download("nishaanamin/march-madness-data")
print("Path to dataset files:", path)

Path to dataset files: /Users/kolbytaylor/.cache/kagglehub/datasets/nishaanamin/march-madness-data/versions/100


In [262]:
# Loading all the different data sets in

# Tournament Results and Data
coach_results = pd.read_csv("march-madness-data/versions/83/Coach Results.csv")
seed_results = pd.read_csv("march-madness-data/versions/83/Seed Results.csv")
team_results = pd.read_csv("march-madness-data/versions/83/Team Results.csv")
conference_results = pd.read_csv("march-madness-data/versions/83/Conference Results.csv")
tournament_locations = pd.read_csv("march-madness-data/versions/83/Tournament Locations.csv")
tournament_matchups = pd.read_csv("march-madness-data/versions/83/Tournament Matchups.csv")
upset_counts = pd.read_csv("march-madness-data/versions/83/Upset Count.csv")
upset_seed_info = pd.read_csv("march-madness-data/versions/83/Upset Seed Info.csv")

# Bartorvik
barttorvik_away_neutral = pd.read_csv("march-madness-data/versions/83/Barttorvik Away-Neutral.csv")
barttorvik_away = pd.read_csv("march-madness-data/versions/83/Barttorvik Away.csv")
barttorvik_home = pd.read_csv("march-madness-data/versions/83/Barttorvik Home.csv")
barttorvik_neutral = pd.read_csv("march-madness-data/versions/83/Barttorvik Home.csv")
kenpom_barttorvik = pd.read_csv("march-madness-data/versions/83/KenPom Barttorvik.csv")

# Conference Stats
conference_stats = pd.read_csv("march-madness-data/versions/83/Conference Stats.csv")
conference_away_neutral = pd.read_csv("march-madness-data/versions/83/Conference Stats Away Neutral.csv")
conference_away = pd.read_csv("march-madness-data/versions/83/Conference Stats Away.csv")
conference_home = pd.read_csv("march-madness-data/versions/83/Conference Stats Home.csv")
conference_neutral = pd.read_csv("march-madness-data/versions/83/Conference Stats Neutral.csv")

# Resumes
resumes = pd.read_csv("march-madness-data/versions/83/Resumes.csv")

# Shooting Splits
shooting_splits = pd.read_csv("march-madness-data/versions/83/Shooting Splits.csv")

# Other Websites
ratings_538 = pd.read_csv("march-madness-data/versions/83/538 Ratings.csv")
heat_check = pd.read_csv("march-madness-data/versions/83/Heat Check Tournament Index.csv")

# People's Opinions
preseason_votes = pd.read_csv("march-madness-data/versions/83/Preseason Votes.csv")
public_picks = pd.read_csv("march-madness-data/versions/83/Public Picks.csv")

In [263]:
# Take the columns in the dataframes that have home, away, and neutral; and subscript them so that we can recognize what it is
# Barttorvik
for col in barttorvik_away_neutral.columns:
    barttorvik_away_neutral = barttorvik_away_neutral.rename(columns={col: col + " AWAY-NEUTRAL"})

for col in barttorvik_away.columns:
    barttorvik_away = barttorvik_away.rename(columns={col: col + " AWAY"})

for col in barttorvik_home.columns:
    barttorvik_home = barttorvik_home.rename(columns={col: col + " HOME"})

for col in barttorvik_neutral.columns:
    barttorvik_neutral = barttorvik_neutral.rename(columns={col: col + " NEUTRAL"})



# Conference Statistics
for col in conference_away_neutral.columns:
    conference_away_neutral = conference_away_neutral.rename(columns={col: col + " AWAY-NEUTRAL"})

for col in conference_away.columns:
    conference_away = conference_away.rename(columns={col: col + " AWAY"})

for col in conference_home.columns:
    conference_home = conference_home.rename(columns={col: col + " HOME"})

for col in conference_neutral.columns:
    conference_neutral = conference_neutral.rename(columns={col: col + " NEUTRAL"})


In [264]:
barttorvik_away_neutral.head()

,YEAR AWAY-NEUTRAL,TEAM NO AWAY-NEUTRAL,TEAM ID AWAY-NEUTRAL,TEAM AWAY-NEUTRAL,SEED AWAY-NEUTRAL,ROUND AWAY-NEUTRAL,BADJ EM AWAY-NEUTRAL,BADJ O AWAY-NEUTRAL,BADJ D AWAY-NEUTRAL,BARTHAG AWAY-NEUTRAL,...,BADJT RANK AWAY-NEUTRAL,AVG HGT RANK AWAY-NEUTRAL,EFF HGT RANK AWAY-NEUTRAL,EXP RANK AWAY-NEUTRAL,TALENT RANK AWAY-NEUTRAL,FT% RANK AWAY-NEUTRAL,OP FT% RANK AWAY-NEUTRAL,PPPO RANK AWAY-NEUTRAL,PPPD RANK AWAY-NEUTRAL,ELITE SOS RANK AWAY-NEUTRAL
0,2024,1079,197,Akron,14,0,3.1,105.0,101.9,0.585,...,265,238,199,19,176,186,188,120,47,269
1,2024,1078,48,Alabama,4,0,18.7,125.4,106.7,0.865,...,10,33,8,156,106,12,357,3,351,8
2,2024,1077,33,Arizona,2,0,23.7,119.7,96.0,0.927,...,40,50,37,196,7,157,284,19,98,52
3,2024,1076,43,Auburn,4,0,26.8,120.8,94.0,0.947,...,67,86,76,127,69,127,260,5,18,77
4,2024,1075,36,Baylor,3,0,17.6,117.7,100.1,0.866,...,275,31,22,304,34,137,248,52,189,11


There are many columns in the Barttorvik data, and splitting them out by where the team played multiplies it by five. In total we have around 170 features. Half of them are metrics and the other half rank those metrics in all Division 1. We should probably sort these out first. I'm going to try doing some analysis with the features. I'm going to fit the same model with the overall, away, netural, home, and neutral-away features and see which ones work the best, as well as do some AB testing to see if there's a significant difference between them.

In [265]:
# Creates the target, 0-6 based on how many games were won
conditions = [
    (kenpom_barttorvik['ROUND'] == 0) | (kenpom_barttorvik['ROUND'] == 68),
    (kenpom_barttorvik['ROUND'] == 64),
    (kenpom_barttorvik['ROUND'] == 32),
    (kenpom_barttorvik['ROUND'] == 16),
    (kenpom_barttorvik['ROUND'] == 8),
    (kenpom_barttorvik['ROUND'] == 4),
    (kenpom_barttorvik['ROUND'] == 2),
    (kenpom_barttorvik['ROUND'] == 1)
]

values = [np.nan, 0, 1, 2, 3, 4, 5, 6]

kenpom_barttorvik['TARGET'] = np.select(conditions, values, default=np.nan)
barttorvik_away_neutral['TARGET'] = np.select(conditions, values, default=np.nan)
barttorvik_away['TARGET'] = np.select(conditions, values, default=np.nan)
barttorvik_home['TARGET'] = np.select(conditions, values, default=np.nan)
barttorvik_neutral['TARGET'] = np.select(conditions, values, default=np.nan)

In [266]:
barttorvik_away_neutral.head()

,YEAR AWAY-NEUTRAL,TEAM NO AWAY-NEUTRAL,TEAM ID AWAY-NEUTRAL,TEAM AWAY-NEUTRAL,SEED AWAY-NEUTRAL,ROUND AWAY-NEUTRAL,BADJ EM AWAY-NEUTRAL,BADJ O AWAY-NEUTRAL,BADJ D AWAY-NEUTRAL,BARTHAG AWAY-NEUTRAL,...,AVG HGT RANK AWAY-NEUTRAL,EFF HGT RANK AWAY-NEUTRAL,EXP RANK AWAY-NEUTRAL,TALENT RANK AWAY-NEUTRAL,FT% RANK AWAY-NEUTRAL,OP FT% RANK AWAY-NEUTRAL,PPPO RANK AWAY-NEUTRAL,PPPD RANK AWAY-NEUTRAL,ELITE SOS RANK AWAY-NEUTRAL,TARGET
0,2024,1079,197,Akron,14,0,3.1,105.0,101.9,0.585,...,238,199,19,176,186,188,120,47,269,NaN
1,2024,1078,48,Alabama,4,0,18.7,125.4,106.7,0.865,...,33,8,156,106,12,357,3,351,8,NaN
2,2024,1077,33,Arizona,2,0,23.7,119.7,96.0,0.927,...,50,37,196,7,157,284,19,98,52,NaN
3,2024,1076,43,Auburn,4,0,26.8,120.8,94.0,0.947,...,86,76,127,69,127,260,5,18,77,NaN
4,2024,1075,36,Baylor,3,0,17.6,117.7,100.1,0.866,...,31,22,304,34,137,248,52,189,11,NaN


In [267]:
X_barttorvik = kenpom_barttorvik.drop(columns=['YEAR', 'CONF', 'CONF ID', 'QUAD NO', 'QUAD ID', 'TEAM NO', 'TEAM ID', 'TEAM', 'SEED', 'ROUND', 'TARGET'])
y_barttorvik = kenpom_barttorvik['TARGET']

X_barttorvik_away_neutral = barttorvik_away_neutral.drop(columns=['YEAR AWAY-NEUTRAL', 'TEAM NO AWAY-NEUTRAL', 'TEAM ID AWAY-NEUTRAL', 'TEAM AWAY-NEUTRAL', 'SEED AWAY-NEUTRAL', 'ROUND AWAY-NEUTRAL', 'TARGET'])
y_barttorvik_away_neutral = barttorvik_away_neutral['TARGET']

X_barttorvik_away = barttorvik_away.drop(columns=['YEAR AWAY', 'TEAM NO AWAY', 'TEAM ID AWAY', 'TEAM AWAY', 'SEED AWAY', 'ROUND AWAY', 'TARGET'])
y_barttorvik_away = barttorvik_away['TARGET']

X_barttorvik_home = barttorvik_home.drop(columns=['YEAR HOME', 'TEAM NO HOME', 'TEAM ID HOME', 'TEAM HOME', 'SEED HOME', 'ROUND HOME', 'TARGET'])
y_barttorvik_home = barttorvik_home['TARGET']

X_barttorvik_neutral = barttorvik_neutral.drop(columns=['YEAR NEUTRAL', 'TEAM NO NEUTRAL', 'TEAM ID NEUTRAL', 'TEAM NEUTRAL', 'SEED NEUTRAL', 'ROUND NEUTRAL', 'TARGET'])
y_barttorvik_neutral = barttorvik_neutral['TARGET']

# Drop rows where TARGET is null
X_barttorvik = X_barttorvik.loc[y_barttorvik.notna()]
y_barttorvik = y_barttorvik.dropna()

X_barttorvik_away_neutral = X_barttorvik_away_neutral.loc[y_barttorvik_away_neutral.notna()]
y_barttorvik_away_neutral = y_barttorvik_away_neutral.dropna()

X_barttorvik_away = X_barttorvik_away.loc[y_barttorvik_away.notna()]
y_barttorvik_away = y_barttorvik_away.dropna()

X_barttorvik_home = X_barttorvik_home.loc[y_barttorvik_home.notna()]
y_barttorvik_home = y_barttorvik_home.dropna()

X_barttorvik_neutral = X_barttorvik_neutral.loc[y_barttorvik_neutral.notna()]
y_barttorvik_neutral = y_barttorvik_neutral.dropna()


I don't want to have to do feature scaling or do anything with assumptions because this is just EDA modeling, so I'm going to use XGBoost.

In [268]:
# No null values! That's nice. 
print(X_barttorvik.isnull().sum()[X_barttorvik.isnull().sum() > 0])

Series([], dtype: int64)


In [269]:
# Datatypes look good
print(X_barttorvik.dtypes)

K TEMPO           float64
K TEMPO RANK        int64
KADJ T            float64
KADJ T RANK         int64
K OFF             float64
                   ...   
FT% RANK            int64
OP FT% RANK         int64
PPPO RANK           int64
PPPD RANK           int64
ELITE SOS RANK      int64
Length: 93, dtype: object


In [270]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_barttorvik, y_barttorvik, test_size=0.2, random_state=42)

# Prepare DMatrix (required for xgb.train)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters grid manually
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 2,
    'learning_rate': 0.2,
    'subsample': 0.6,
    'alpha': 10,
    'colsample_bytree': 0.6,
    'random_state': 42,
    'min_child_weight': 5
}

# Set up training with early stopping
evallist = [(dtest, 'eval')]

# Train the model with early stopping
bst = xgb.train(
    params, 
    dtrain, 
    num_boost_round=1000,
    evals=evallist, 
    early_stopping_rounds=20,
    verbose_eval=10
)

# Get the best iteration from early stopping
best_iteration = bst.best_iteration
print(f"Best iteration: {best_iteration}")

# Make predictions using the best iteration
y_pred = bst.predict(dtest, iteration_range=(0, best_iteration))

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the result
print(f"Test RMSE: {test_rmse:.4f}")


[0]	eval-rmse:1.24614
[10]	eval-rmse:1.07732
[20]	eval-rmse:1.08317
[30]	eval-rmse:1.08900
[33]	eval-rmse:1.09478
Best iteration: 14
Test RMSE: 1.0766


In [271]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_barttorvik_away_neutral, y_barttorvik_away_neutral, test_size=0.2, random_state=42)

# Prepare DMatrix (required for xgb.train)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters grid manually
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 2,
    'learning_rate': 0.2,
    'subsample': 0.6,
    'alpha': 10,
    'colsample_bytree': 0.6,
    'random_state': 42,
    'min_child_weight': 5
}

# Set up training with early stopping
evallist = [(dtest, 'eval')]

# Train the model with early stopping
bst = xgb.train(
    params, 
    dtrain, 
    num_boost_round=1000,
    evals=evallist, 
    early_stopping_rounds=20,
    verbose_eval=10
)

# Get the best iteration from early stopping
best_iteration = bst.best_iteration
print(f"Best iteration: {best_iteration}")

# Make predictions using the best iteration
y_pred = bst.predict(dtest, iteration_range=(0, best_iteration))

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the result
print(f"Test RMSE: {test_rmse:.4f}")


[0]	eval-rmse:1.24819
[10]	eval-rmse:1.08775
[20]	eval-rmse:1.08954
[30]	eval-rmse:1.09811
[33]	eval-rmse:1.10664
Best iteration: 13
Test RMSE: 1.0897


In [272]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_barttorvik_away, y_barttorvik_away, test_size=0.2, random_state=42)

# Prepare DMatrix (required for xgb.train)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters grid manually
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 2,
    'learning_rate': 0.2,
    'subsample': 0.6,
    'alpha': 10,
    'colsample_bytree': 0.6,
    'random_state': 42,
    'min_child_weight': 5
}

# Set up training with early stopping
evallist = [(dtest, 'eval')]

# Train the model with early stopping
bst = xgb.train(
    params, 
    dtrain, 
    num_boost_round=1000,
    evals=evallist, 
    early_stopping_rounds=20,
    verbose_eval=10
)

# Get the best iteration from early stopping
best_iteration = bst.best_iteration
print(f"Best iteration: {best_iteration}")

# Make predictions using the best iteration
y_pred = bst.predict(dtest, iteration_range=(0, best_iteration))

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the result
print(f"Test RMSE: {test_rmse:.4f}")


[0]	eval-rmse:1.24884
[10]	eval-rmse:1.12063
[20]	eval-rmse:1.11433
[30]	eval-rmse:1.12300
[33]	eval-rmse:1.13158
Best iteration: 14
Test RMSE: 1.1112


In [273]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_barttorvik_home, y_barttorvik_home, test_size=0.2, random_state=42)

# Prepare DMatrix (required for xgb.train)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters grid manually
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 2,
    'learning_rate': 0.2,
    'subsample': 0.6,
    'alpha': 10,
    'colsample_bytree': 0.6,
    'random_state': 42,
    'min_child_weight': 5
}

# Set up training with early stopping
evallist = [(dtest, 'eval')]

# Train the model with early stopping
bst = xgb.train(
    params, 
    dtrain, 
    num_boost_round=1000,
    evals=evallist, 
    early_stopping_rounds=20,
    verbose_eval=10
)

# Get the best iteration from early stopping
best_iteration = bst.best_iteration
print(f"Best iteration: {best_iteration}")

# Make predictions using the best iteration
y_pred = bst.predict(dtest, iteration_range=(0, best_iteration))

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the result
print(f"Test RMSE: {test_rmse:.4f}")


[0]	eval-rmse:1.24086
[10]	eval-rmse:1.07804
[20]	eval-rmse:1.08300
[30]	eval-rmse:1.09856
Best iteration: 10
Test RMSE: 1.0840


In [274]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X_barttorvik_neutral, y_barttorvik_neutral, test_size=0.2, random_state=42)

# Prepare DMatrix (required for xgb.train)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define hyperparameters grid manually
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 2,
    'learning_rate': 0.2,
    'subsample': 0.6,
    'alpha': 10,
    'colsample_bytree': 0.6,
    'random_state': 42,
    'min_child_weight': 5
}

# Set up training with early stopping
evallist = [(dtest, 'eval')]

# Train the model with early stopping
bst = xgb.train(
    params, 
    dtrain, 
    num_boost_round=1000,
    evals=evallist, 
    early_stopping_rounds=20,
    verbose_eval=10
)

# Get the best iteration from early stopping
best_iteration = bst.best_iteration
print(f"Best iteration: {best_iteration}")

# Make predictions using the best iteration
y_pred = bst.predict(dtest, iteration_range=(0, best_iteration))

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the result
print(f"Test RMSE: {test_rmse:.4f}")


[0]	eval-rmse:1.24086
[10]	eval-rmse:1.07804
[20]	eval-rmse:1.08300
[29]	eval-rmse:1.09671
Best iteration: 10
Test RMSE: 1.0840


### Model Results:

The results were consistent across all models. The hyperparameters selected by each model were identical:

- subsample: 0.6  
- min_child_weight: 5  
- max_depth: 2  
- learning_rate: 0.05  
- lambda: 5  
- colsample_bytree: 1.0  
- alpha: 10  

---

### RMSE:

- Overall: 1.07  
- Away-Neutral: 1.08  
- Away: 1.11  
- Home: 1.08  
- Neutral: 1.08  

---

The Overall RMSE might be slightly better due to the inclusion of a few different features. Based on this analysis, we have decided to use the overall Barttorvik data in our analysis moving forward.


In [275]:
# Starting with the Kenpom data
all_data = kenpom_barttorvik

# Add 538 Ratings
all_data = all_data.merge(ratings_538, how='left', on='TEAM NO')
all_data = all_data.drop(columns=['YEAR_y', 'TEAM_y', 'SEED_y', 'ROUND_y'])
all_data = all_data.rename(columns={'YEAR_x': 'YEAR', 'TEAM_x': 'TEAM', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND'})

# Add Heat Check metrics
all_data = all_data.merge(heat_check, how='left', on='TEAM NO')
all_data = all_data.drop(columns=['YEAR_y', 'TEAM_y', 'SEED_y', 'ROUND_y'])
all_data = all_data.rename(columns={'YEAR_x': 'YEAR', 'TEAM_x': 'TEAM', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND'})

# Add resume data, includes strength of schedule
all_data = all_data.merge(resumes, how='left', on='TEAM NO')
all_data = all_data.drop(columns=['YEAR_y', 'TEAM_y'])
all_data = all_data.rename(columns={'YEAR_x': 'YEAR', 'TEAM_x': 'TEAM', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND'})

# Shooting splits of each team
all_data = all_data.merge(shooting_splits, how='left', on='TEAM NO')
all_data = all_data.drop(columns=['YEAR_y', 'TEAM_y', 'SEED_y', 'ROUND_y'])
all_data = all_data.rename(columns={'YEAR_x': 'YEAR', 'TEAM_x': 'TEAM', 'SEED_x': 'SEED', 'ROUND_x': 'ROUND', 'CONF_x': 'CONF', 'TEAM ID_x': 'TEAM ID'})

# Drops any more columns that don't seem useful
all_data = all_data.drop(columns=['CONF ID', 'QUAD NO', 'QUAD ID', 'TEAM NO', 'TEAM ID', 'TARGET', 'VAL Z-SCORE'])

There are coach and conference results that seem cool and would be interesting, but scare me because of potential data leakage. For example, Bill Self won a championship a few years ago, but before that his championships would be at zero... So I don't think it's a good idea to include it. Conference results are similar, but might be more watered down because they're average over the entire conference.

In [276]:
# Prints the number of columns
print(len(all_data.columns))

154


Our feature space is really high, so we're going to try and reduce it by removing the columns that rank the features. These are obviously highly correlated with the features themselves, we're just losing some information about how the team compared to other teams in its season.

In [277]:
all_data = all_data.drop(columns=[col for col in all_data.columns if 'RANK' in col])

In [278]:
# Prints the number of columns
print(len(all_data.columns))

90


Okay, now we're down to a more reasonable space. I'm sure there's some unimportant features, or features with high collinearity that we can weed out, but we'll have to more to find them.

In [279]:
all_data.to_csv("All_Data.csv", index=False)

In [280]:
all_data

,YEAR,CONF,TEAM,SEED,ROUND,K TEMPO,KADJ T,K OFF,KADJ O,K DEF,...,CLOSE TWOS FG%D,CLOSE TWOS D SHARE,FARTHER TWOS FG%,FARTHER TWOS SHARE,FARTHER TWOS FG%D,FARTHER TWOS D SHARE,THREES FG%,THREES SHARE,THREES FG%D,THREES D SHARE
0,2024,MAC,Akron,14,0,66.7747,65.8933,107.841,107.0090,99.7096,...,58.0,37.7,43.8,24.6,39.9,25.6,32.0,41.5,30.0,36.8
1,2024,SEC,Alabama,4,0,74.1625,72.6461,121.712,125.6010,108.0310,...,60.2,37.9,35.7,10.8,37.6,25.3,36.5,46.8,31.9,36.7
2,2024,P12,Arizona,2,0,73.3760,71.8379,117.653,121.1250,96.9086,...,57.4,30.3,37.9,23.1,38.5,31.5,37.1,32.6,33.4,38.2
3,2024,SEC,Auburn,4,0,70.9629,69.7887,117.364,120.5790,95.7924,...,48.3,42.0,41.6,21.1,33.6,24.7,35.2,37.5,29.8,33.3
4,2024,B12,Baylor,3,0,66.8428,65.6032,117.262,122.4900,104.1660,...,65.7,29.8,37.5,26.9,40.0,34.6,39.4,38.3,33.5,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,2008,BE,West Virginia,7,16,66.2223,63.0942,111.246,113.5700,95.1209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,2008,SB,Western Kentucky,12,16,68.7887,66.5432,110.914,111.0760,94.3478,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1076,2008,BSth,Winthrop,13,64,63.7666,60.9746,101.646,99.6932,92.1731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1077,2008,B10,Wisconsin,3,16,62.1920,60.8809,108.495,113.6190,86.5633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# EDA